In [1]:
# @FWS-IMPORTS add all import requirements here
from radlib_imports import *
# from radlib.idia.idia_dicom_converter import DicomConversion
from radlib.fw.flywheel_clients import uwhealthaz_client
from radlib.fws.fws_utils import fws_input_file_list, fws_load_image, fws_in_jupyter_notebook
from radlib.fws.fws_image import FWSImageFile, FWSImageFileList
from radlib.idia.idia_dicom_selection import DicomSelectionSingle, convert_dataset_to_nifti, mri_data_check

In [2]:
# main_config.yaml
main_config = yaml.safe_load('''
# root_path: "/app/Glioma" # data folder (parent)
root_path: "//onfnas01.uwhis.hosp.wisc.edu/radiology/Groups/IDIAGroup/Data/_Brain/Radiology/_Adult/_Glioma"
#root_path: "/app/Data/_Brain/Radiology/_Adult/_Glioma" # data folder (parent)
#root_path: "Y:/Groups/IDIAGroup/Data/_Brain/Radiology/_Adult/_Glioma"

mri_sites:  # add as many dataset as nedeed (coded by site{n})
  site0: "UCSF-PDGM"
  site1: "LUMIERE"
  site2: "CCF"
  site3: "IVYGAP"
  site4: "xCures"
  site5: "TCGA-GBM"
  site6: "GBM_Cohort_brucegroup/UW-GBM"

mri_data: ["site6"] # data to be included in the pipeline

submodule_configs:
  preprocessing: "config/preprocessing_config.yaml"
  radiomics: "config/radiomics_config.yaml"

# Enabling modules
run_dicomSelection: False
run_dicomConversion: False
run_niftiSelection: False
run_preprocessing: True
run_segmentation: False
run_radiomics: False
run_survival_train: False
run_survival_test: False
run_deep_learning: False
run_feature_visualization: False
''')

In [3]:
# preprocessing_config.yaml
preprocessing_config = yaml.safe_load('''
dirs:
  raw_dicoms: "dicom_raw"
  input_dicoms: "dicom_structuralMRI_csk"
  raw_nifti: "nifti_raw_csk"
  preprocessed: "preprocessed_csk"
  metadata: "Metadata_csk"
  logs: "logs_csk"

#pre-processign pipelines / parameters
 
preprocessing_settings:
  query_nifti_file: "niiQuery.csv"
  acquisition_tag: "Baseline"
  query_key: "included_modality"
  mri_modalities: ["T1c","T1","T2","FLAIR"]

ants_preprocessing:
  first_step: 
    name: "Modality_2_atlas_reg"
    mri_str: "T1c" 
    atlas_str: "SRI24"
    ext: ".nii.gz"
    mask_str: None
    aff_metric : "mattes"
    type_of_transform : "Affine"
    transforming_mask: False
    brain_mask: None # HD_BET or Atlas_mask Default= None

  second_step: 
    name: "Modalities_coregistration"
    ext: ".nii.gz"
    aff_metric : "mattes"
    type_of_transform : "Affine"
    transforming_mask: False
    SkullStripp: None # HD_BET or Atlas_mask Default= None

  third_step: 
    name: "HD_SkullStripp"

  fourth_step: 
    name: "N4Bias_correction"
  fifth_step:  
    name: "IntensityScaling"

# image_resize:
#   first_step: "T1c_coregistration"
#   second_step:  "HD_SkullStripp"
#   third_step: "N4Bias_correction"
#   fourth_step:  "ants_resampling"
''')

In [4]:
# radiomics_config.yaml
radiomics_config = yaml.safe_load('''
analysis_method: "radiomics"
features_to_extract: ["shape", "texture", "intensity"]
''')

In [5]:
# @FWS-FILES part of the script mechanism that will be replaced by the gear"s context
root_path="z:/radiology/Groups/IDIAGroup/Data/_Brain/Radiology/_Adult/_Glioma/GBM_Cohort_brucegroup/"
mri_data="TCGA-GBM"
indir="DICOM_structuralMRI_fws"
outdir="nifti_raw"

input_path=os.path.join(root_path,mri_data,indir)
output_path=os.path.join(root_path,mri_data,outdir)

fw_client = uwhealthaz_client()
group_id = ''
project_label = "GBM Cohort"
subject_label = "RAD-AI-CNS-TUMOR-0001"

logger = logging.getLogger(__name__)
fws_files = fws_input_file_list(fw_client, project_label, subject_labels=['RAD-AI-CNS-TUMOR-0001'], generate_code=False,
                                        local_root='//onfnas01.uwhis.hosp.wisc.edu/radiology/Groups/IDIAGroup/Data/_Brain/Radiology/_Adult/_Glioma/GBM_Cohort_brucegroup/UW-GBM/dicom_raw/RAD-AI-CNS-TUMOR-0001/2019-12-10 09_30_33')

In [6]:
# FWS-INPUTS edit this cell to the flywheel file references that you need, add local_path(s) if necessary, then TURN OFF generate_code above
local_root = "//onfnas01.uwhis.hosp.wisc.edu/radiology/Groups/IDIAGroup/Data/_Brain/Radiology/_Adult/_Glioma/GBM_Cohort_brucegroup/UW-GBM/dicom_raw/RAD-AI-CNS-TUMOR-0001/2019-12-10 09_30_33"
fws_input_files = FWSImageFileList( {
"2019_12_10_09_30_33_1 - 3-Plane Loc.dicom.zip": FWSImageFile(fw_client=fw_client, fw_path="brucegroup/GBM Cohort/RAD-AI-CNS-TUMOR-0001/2019-12-10 09_30_33/1 - 3-Plane Loc/1 - 3-Plane Loc.dicom.zip",
	local_path="//onfnas01.uwhis.hosp.wisc.edu/radiology/Groups/IDIAGroup/Data/_Brain/Radiology/_Adult/_Glioma/GBM_Cohort_brucegroup/UW-GBM/dicom_raw/RAD-AI-CNS-TUMOR-0001/2019-12-10 09_30_33/1 - 3-Plane Loc.dicom.zip"),
"2019_12_10_09_30_33_10 - Ax T2star GRE.dicom.zip": FWSImageFile(fw_client=fw_client, fw_path="brucegroup/GBM Cohort/RAD-AI-CNS-TUMOR-0001/2019-12-10 09_30_33/10 - Ax T2star GRE/10 - Ax T2star GRE.dicom.zip",
	local_path="//onfnas01.uwhis.hosp.wisc.edu/radiology/Groups/IDIAGroup/Data/_Brain/Radiology/_Adult/_Glioma/GBM_Cohort_brucegroup/UW-GBM/dicom_raw/RAD-AI-CNS-TUMOR-0001/2019-12-10 09_30_33/10 - Ax T2star GRE.dicom.zip"),
"2019_12_10_09_30_33_11 - PseudoSwan.dicom.zip": FWSImageFile(fw_client=fw_client, fw_path="brucegroup/GBM Cohort/RAD-AI-CNS-TUMOR-0001/2019-12-10 09_30_33/11 - PseudoSwan/11 - PseudoSwan.dicom.zip",
	local_path="//onfnas01.uwhis.hosp.wisc.edu/radiology/Groups/IDIAGroup/Data/_Brain/Radiology/_Adult/_Glioma/GBM_Cohort_brucegroup/UW-GBM/dicom_raw/RAD-AI-CNS-TUMOR-0001/2019-12-10 09_30_33/11 - PseudoSwan.dicom.zip"),
"2019_12_10_09_30_33_1199 - Perfusion ROIs.dicom.zip": FWSImageFile(fw_client=fw_client, fw_path="brucegroup/GBM Cohort/RAD-AI-CNS-TUMOR-0001/2019-12-10 09_30_33/1199 - Perfusion ROIs/1199 - Perfusion ROIs.dicom.zip",
	local_path="//onfnas01.uwhis.hosp.wisc.edu/radiology/Groups/IDIAGroup/Data/_Brain/Radiology/_Adult/_Glioma/GBM_Cohort_brucegroup/UW-GBM/dicom_raw/RAD-AI-CNS-TUMOR-0001/2019-12-10 09_30_33/1199 - Perfusion ROIs.dicom.zip"),
"2019_12_10_09_30_33_12 - TUMOR-Ax PERFUSION.dicom.zip": FWSImageFile(fw_client=fw_client, fw_path="brucegroup/GBM Cohort/RAD-AI-CNS-TUMOR-0001/2019-12-10 09_30_33/12 - TUMOR-Ax PERFUSION/12 - TUMOR-Ax PERFUSION.dicom.zip",
	local_path="//onfnas01.uwhis.hosp.wisc.edu/radiology/Groups/IDIAGroup/Data/_Brain/Radiology/_Adult/_Glioma/GBM_Cohort_brucegroup/UW-GBM/dicom_raw/RAD-AI-CNS-TUMOR-0001/2019-12-10 09_30_33/12 - TUMOR-Ax PERFUSION.dicom.zip"),
"2019_12_10_09_30_33_1200 - Baseline.dicom.zip": FWSImageFile(fw_client=fw_client, fw_path="brucegroup/GBM Cohort/RAD-AI-CNS-TUMOR-0001/2019-12-10 09_30_33/1200 - Baseline/1200 - Baseline.dicom.zip",
	local_path="//onfnas01.uwhis.hosp.wisc.edu/radiology/Groups/IDIAGroup/Data/_Brain/Radiology/_Adult/_Glioma/GBM_Cohort_brucegroup/UW-GBM/dicom_raw/RAD-AI-CNS-TUMOR-0001/2019-12-10 09_30_33/1200 - Baseline.dicom.zip"),
"2019_12_10_09_30_33_1202 - rCBV (corrected)(color).dicom.zip": FWSImageFile(fw_client=fw_client, fw_path="brucegroup/GBM Cohort/RAD-AI-CNS-TUMOR-0001/2019-12-10 09_30_33/1202 - rCBV (corrected)(color)/1202 - rCBV (corrected)(color).dicom.zip",
	local_path="//onfnas01.uwhis.hosp.wisc.edu/radiology/Groups/IDIAGroup/Data/_Brain/Radiology/_Adult/_Glioma/GBM_Cohort_brucegroup/UW-GBM/dicom_raw/RAD-AI-CNS-TUMOR-0001/2019-12-10 09_30_33/1202 - rCBV (corrected)(color).dicom.zip"),
"2019_12_10_09_30_33_1203 - CBF (color).dicom.zip": FWSImageFile(fw_client=fw_client, fw_path="brucegroup/GBM Cohort/RAD-AI-CNS-TUMOR-0001/2019-12-10 09_30_33/1203 - CBF (color)/1203 - CBF (color).dicom.zip",
	local_path="//onfnas01.uwhis.hosp.wisc.edu/radiology/Groups/IDIAGroup/Data/_Brain/Radiology/_Adult/_Glioma/GBM_Cohort_brucegroup/UW-GBM/dicom_raw/RAD-AI-CNS-TUMOR-0001/2019-12-10 09_30_33/1203 - CBF (color).dicom.zip"),
"2019_12_10_09_30_33_1204 - FMT (color).dicom.zip": FWSImageFile(fw_client=fw_client, fw_path="brucegroup/GBM Cohort/RAD-AI-CNS-TUMOR-0001/2019-12-10 09_30_33/1204 - FMT (color)/1204 - FMT (color).dicom.zip",
	local_path="//onfnas01.uwhis.hosp.wisc.edu/radiology/Groups/IDIAGroup/Data/_Brain/Radiology/_Adult/_Glioma/GBM_Cohort_brucegroup/UW-GBM/dicom_raw/RAD-AI-CNS-TUMOR-0001/2019-12-10 09_30_33/1204 - FMT (color).dicom.zip"),
"2019_12_10_09_30_33_1205 - Tmax (color).dicom.zip": FWSImageFile(fw_client=fw_client, fw_path="brucegroup/GBM Cohort/RAD-AI-CNS-TUMOR-0001/2019-12-10 09_30_33/1205 - Tmax (color)/1205 - Tmax (color).dicom.zip",
	local_path="//onfnas01.uwhis.hosp.wisc.edu/radiology/Groups/IDIAGroup/Data/_Brain/Radiology/_Adult/_Glioma/GBM_Cohort_brucegroup/UW-GBM/dicom_raw/RAD-AI-CNS-TUMOR-0001/2019-12-10 09_30_33/1205 - Tmax (color).dicom.zip"),
"2019_12_10_09_30_33_1206 - K2 (color).dicom.zip": FWSImageFile(fw_client=fw_client, fw_path="brucegroup/GBM Cohort/RAD-AI-CNS-TUMOR-0001/2019-12-10 09_30_33/1206 - K2 (color)/1206 - K2 (color).dicom.zip",
	local_path="//onfnas01.uwhis.hosp.wisc.edu/radiology/Groups/IDIAGroup/Data/_Brain/Radiology/_Adult/_Glioma/GBM_Cohort_brucegroup/UW-GBM/dicom_raw/RAD-AI-CNS-TUMOR-0001/2019-12-10 09_30_33/1206 - K2 (color).dicom.zip"),
"2019_12_10_09_30_33_1212 - rCBV (corrected).dicom.zip": FWSImageFile(fw_client=fw_client, fw_path="brucegroup/GBM Cohort/RAD-AI-CNS-TUMOR-0001/2019-12-10 09_30_33/1212 - rCBV (corrected)/1212 - rCBV (corrected).dicom.zip",
	local_path="//onfnas01.uwhis.hosp.wisc.edu/radiology/Groups/IDIAGroup/Data/_Brain/Radiology/_Adult/_Glioma/GBM_Cohort_brucegroup/UW-GBM/dicom_raw/RAD-AI-CNS-TUMOR-0001/2019-12-10 09_30_33/1212 - rCBV (corrected).dicom.zip"),
"2019_12_10_09_30_33_1216 - K2.dicom.zip": FWSImageFile(fw_client=fw_client, fw_path="brucegroup/GBM Cohort/RAD-AI-CNS-TUMOR-0001/2019-12-10 09_30_33/1216 - K2/1216 - K2.dicom.zip",
	local_path="//onfnas01.uwhis.hosp.wisc.edu/radiology/Groups/IDIAGroup/Data/_Brain/Radiology/_Adult/_Glioma/GBM_Cohort_brucegroup/UW-GBM/dicom_raw/RAD-AI-CNS-TUMOR-0001/2019-12-10 09_30_33/1216 - K2.dicom.zip"),
"2019_12_10_09_30_33_13 - +C Sag T2 FLAIR.dicom.zip": FWSImageFile(fw_client=fw_client, fw_path="brucegroup/GBM Cohort/RAD-AI-CNS-TUMOR-0001/2019-12-10 09_30_33/13 - +C Sag T2 FLAIR/13 - +C Sag T2 FLAIR.dicom.zip",
	local_path="//onfnas01.uwhis.hosp.wisc.edu/radiology/Groups/IDIAGroup/Data/_Brain/Radiology/_Adult/_Glioma/GBM_Cohort_brucegroup/UW-GBM/dicom_raw/RAD-AI-CNS-TUMOR-0001/2019-12-10 09_30_33/13 - +C Sag T2 FLAIR.dicom.zip"),
"2019_12_10_09_30_33_14 - +C Ax T1 BRAVO Stealth.dicom.zip": FWSImageFile(fw_client=fw_client, fw_path="brucegroup/GBM Cohort/RAD-AI-CNS-TUMOR-0001/2019-12-10 09_30_33/14 - +C Ax T1 BRAVO Stealth/14 - +C Ax T1 BRAVO Stealth.dicom.zip",
	local_path="//onfnas01.uwhis.hosp.wisc.edu/radiology/Groups/IDIAGroup/Data/_Brain/Radiology/_Adult/_Glioma/GBM_Cohort_brucegroup/UW-GBM/dicom_raw/RAD-AI-CNS-TUMOR-0001/2019-12-10 09_30_33/14 - +C Ax T1 BRAVO Stealth.dicom.zip"),
"2019_12_10_09_30_33_15 - +C Cor T1 Cube VASC.dicom.zip": FWSImageFile(fw_client=fw_client, fw_path="brucegroup/GBM Cohort/RAD-AI-CNS-TUMOR-0001/2019-12-10 09_30_33/15 - +C Cor T1 Cube VASC/15 - +C Cor T1 Cube VASC.dicom.zip",
	local_path="//onfnas01.uwhis.hosp.wisc.edu/radiology/Groups/IDIAGroup/Data/_Brain/Radiology/_Adult/_Glioma/GBM_Cohort_brucegroup/UW-GBM/dicom_raw/RAD-AI-CNS-TUMOR-0001/2019-12-10 09_30_33/15 - +C Cor T1 Cube VASC.dicom.zip"),
"2019_12_10_09_30_33_16 - +C Cor T1 Fat FSPGR.dicom.zip": FWSImageFile(fw_client=fw_client, fw_path="brucegroup/GBM Cohort/RAD-AI-CNS-TUMOR-0001/2019-12-10 09_30_33/16 - +C Cor T1 Fat FSPGR/16 - +C Cor T1 Fat FSPGR.dicom.zip",
	local_path="//onfnas01.uwhis.hosp.wisc.edu/radiology/Groups/IDIAGroup/Data/_Brain/Radiology/_Adult/_Glioma/GBM_Cohort_brucegroup/UW-GBM/dicom_raw/RAD-AI-CNS-TUMOR-0001/2019-12-10 09_30_33/16 - +C Cor T1 Fat FSPGR.dicom.zip"),
"2019_12_10_09_30_33_3 - Sag T1 FLAIR.dicom.zip": FWSImageFile(fw_client=fw_client, fw_path="brucegroup/GBM Cohort/RAD-AI-CNS-TUMOR-0001/2019-12-10 09_30_33/3 - Sag T1 FLAIR/3 - Sag T1 FLAIR.dicom.zip",
	local_path="//onfnas01.uwhis.hosp.wisc.edu/radiology/Groups/IDIAGroup/Data/_Brain/Radiology/_Adult/_Glioma/GBM_Cohort_brucegroup/UW-GBM/dicom_raw/RAD-AI-CNS-TUMOR-0001/2019-12-10 09_30_33/3 - Sag T1 FLAIR.dicom.zip"),
"2019_12_10_09_30_33_4 - Ax T2 FRFSE FAT.dicom.zip": FWSImageFile(fw_client=fw_client, fw_path="brucegroup/GBM Cohort/RAD-AI-CNS-TUMOR-0001/2019-12-10 09_30_33/4 - Ax T2 FRFSE FAT/4 - Ax T2 FRFSE FAT.dicom.zip",
	local_path="//onfnas01.uwhis.hosp.wisc.edu/radiology/Groups/IDIAGroup/Data/_Brain/Radiology/_Adult/_Glioma/GBM_Cohort_brucegroup/UW-GBM/dicom_raw/RAD-AI-CNS-TUMOR-0001/2019-12-10 09_30_33/4 - Ax T2 FRFSE FAT.dicom.zip"),
"2019_12_10_09_30_33_5 - Ax DWI.dicom.zip": FWSImageFile(fw_client=fw_client, fw_path="brucegroup/GBM Cohort/RAD-AI-CNS-TUMOR-0001/2019-12-10 09_30_33/5 - Ax DWI/5 - Ax DWI.dicom.zip",
	local_path="//onfnas01.uwhis.hosp.wisc.edu/radiology/Groups/IDIAGroup/Data/_Brain/Radiology/_Adult/_Glioma/GBM_Cohort_brucegroup/UW-GBM/dicom_raw/RAD-AI-CNS-TUMOR-0001/2019-12-10 09_30_33/5 - Ax DWI.dicom.zip"),
"2019_12_10_09_30_33_550 - ADC.dicom.zip": FWSImageFile(fw_client=fw_client, fw_path="brucegroup/GBM Cohort/RAD-AI-CNS-TUMOR-0001/2019-12-10 09_30_33/550 - ADC/550 - ADC.dicom.zip",
	local_path="//onfnas01.uwhis.hosp.wisc.edu/radiology/Groups/IDIAGroup/Data/_Brain/Radiology/_Adult/_Glioma/GBM_Cohort_brucegroup/UW-GBM/dicom_raw/RAD-AI-CNS-TUMOR-0001/2019-12-10 09_30_33/550 - ADC.dicom.zip"),
"2019_12_10_09_30_33_6 - Ax T2star GRE.dicom.zip": FWSImageFile(fw_client=fw_client, fw_path="brucegroup/GBM Cohort/RAD-AI-CNS-TUMOR-0001/2019-12-10 09_30_33/6 - Ax T2star GRE/6 - Ax T2star GRE.dicom.zip",
	local_path="//onfnas01.uwhis.hosp.wisc.edu/radiology/Groups/IDIAGroup/Data/_Brain/Radiology/_Adult/_Glioma/GBM_Cohort_brucegroup/UW-GBM/dicom_raw/RAD-AI-CNS-TUMOR-0001/2019-12-10 09_30_33/6 - Ax T2star GRE.dicom.zip"),
"2019_12_10_09_30_33_7 - Cor T2 Fat.dicom.zip": FWSImageFile(fw_client=fw_client, fw_path="brucegroup/GBM Cohort/RAD-AI-CNS-TUMOR-0001/2019-12-10 09_30_33/7 - Cor T2 Fat/7 - Cor T2 Fat.dicom.zip",
	local_path="//onfnas01.uwhis.hosp.wisc.edu/radiology/Groups/IDIAGroup/Data/_Brain/Radiology/_Adult/_Glioma/GBM_Cohort_brucegroup/UW-GBM/dicom_raw/RAD-AI-CNS-TUMOR-0001/2019-12-10 09_30_33/7 - Cor T2 Fat.dicom.zip"),
"2019_12_10_09_30_33_8 - Cor T1 3D BRAVO.dicom.zip": FWSImageFile(fw_client=fw_client, fw_path="brucegroup/GBM Cohort/RAD-AI-CNS-TUMOR-0001/2019-12-10 09_30_33/8 - Cor T1 3D BRAVO/8 - Cor T1 3D BRAVO.dicom.zip",
	local_path="//onfnas01.uwhis.hosp.wisc.edu/radiology/Groups/IDIAGroup/Data/_Brain/Radiology/_Adult/_Glioma/GBM_Cohort_brucegroup/UW-GBM/dicom_raw/RAD-AI-CNS-TUMOR-0001/2019-12-10 09_30_33/8 - Cor T1 3D BRAVO.dicom.zip"),
"2019_12_10_09_30_33_9 - 3D Ax ASL.dicom.zip": FWSImageFile(fw_client=fw_client, fw_path="brucegroup/GBM Cohort/RAD-AI-CNS-TUMOR-0001/2019-12-10 09_30_33/9 - 3D Ax ASL/9 - 3D Ax ASL.dicom.zip",
	local_path="//onfnas01.uwhis.hosp.wisc.edu/radiology/Groups/IDIAGroup/Data/_Brain/Radiology/_Adult/_Glioma/GBM_Cohort_brucegroup/UW-GBM/dicom_raw/RAD-AI-CNS-TUMOR-0001/2019-12-10 09_30_33/9 - 3D Ax ASL.dicom.zip"),
"2019_12_10_09_30_33_2 - Cal Head 34.dicom.zip": FWSImageFile(fw_client=fw_client, fw_path="brucegroup/GBM Cohort/RAD-AI-CNS-TUMOR-0001/2019-12-10 09_30_33/2 - Cal Head 34/2 - Cal Head 34.dicom.zip",
	local_path="//onfnas01.uwhis.hosp.wisc.edu/radiology/Groups/IDIAGroup/Data/_Brain/Radiology/_Adult/_Glioma/GBM_Cohort_brucegroup/UW-GBM/dicom_raw/RAD-AI-CNS-TUMOR-0001/2019-12-10 09_30_33/2 - Cal Head 34.dicom.zip"),
})

In [7]:
# @FWS-STEP 1 dicom selection
logger.info(f"{mri_data} Starting dicom selection")
selected_files = DicomSelectionSingle(fws_input_files)

time_point>>>> RAD-AI-CNS-TUMOR-0001


In [8]:
#  @FWS-STEP 2 dicom conversion
logger.info(f"Starting dicom conversion")
convert_dataset_to_nifti(selected_files, fw_acquisition_label='nifti_raw')
logger.info(f" DICOM CONVERSION DONE")

In [10]:
def mri_data_check(fws_selected_files):
    for file in fws_selected_files.values():
        print(file.file_name())

# @FWS-STEP 3 nifti selection
logger.info(f" Starting Nifti selection for ")

# MRIQc=MRI_DataCheck(dataset, dirs.raw_nifti, root_path, dirs.logs) # from utils.generic
mri_data_check(selected_files)
# MRIQc.copy_and_rename_files(pp_settings, dirs.raw_nifti, dirs.preprocessed)

logger.info(f" Nifti SELECTION DONE for")


Ax T2star GRE.nii.gz
C Sag T2 FLAIR.nii.gz
C Ax T1 BRAVO Stealth.nii.gz
C Cor T1 Fat FSPGR.nii.gz
Sag T1 FLAIR.nii.gz
Ax T2 FRFSE FAT.nii.gz
Ax T2star GRE.nii.gz
Cor T2 Fat.nii.gz
Cor T1 3D BRAVO.nii.gz


In [ ]:
# @FWS-STEP 4 preprocessing
logger.info(f" Starting MRI Preprocessing for {dataset}")

pp_config = Config2Struct(configs["preprocessing"]["ants_preprocessing"])
#get_config_params(pp_config) # print Config attributtes

MRIpp=MRIPreprocessing(dataset, dirs, gen_config, pp_settings, pp_config) # from preprocessing.ants_pp_utils

MRIpp.run_pipeline()


In [ ]:
# @FWS-STEP 5 verify conversion
verify_conversion(input_path, output_path)
# old results:
# Total DICOM series: 541
# Total NIfTI files created: 536
# Conversion rate: 99.08%